<a href="https://colab.research.google.com/github/network-evolution/GenerativeAI_Tutorial_Notebooks/blob/main/03_02_stock_price_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.3 MB/s eta 0:00:00


In [ ]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
added 22 packages in 5s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/colab_files/stock_price_app'

Mounted at /content/drive


In [ ]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('openai')

In [ ]:
!streamlit run {base_path}/app.py &>/content/st_logs.txt &

In [ ]:
!npx localtunnel --port 8501

⠙your url is: https://cool-lights-tell.loca.lt


In [ ]:
from openai import OpenAI
import json
client = OpenAI()


def get_stock_details(user_query):
  tools = [{
    "type": "function",
    "name": "get_stock_price",
    "description": "Get current or historic stock price for a given company name or ticker symbol.",
    "parameters": {
        "type": "object",
        "properties": {
            "ticker_symbol": {
                "type": "string",
                "description": "Stock ticker symbol like AAPL, TSLA, MSFT, INFY.NS, INFY.BO,"
            },
            "date": {
                "type": "string",
                "description": "Date in YYYY-MM-DD format to get the stock price. If not provided, defaults to today."
            }
        },
        "required": ["ticker_symbol"],
        "additionalProperties": False
    }
  }]
  input_messages = [{"role": "user", "content": user_query}]
  response = client.responses.create(
      model="gpt-4o",
      input=input_messages,
      tools=tools,
  )
  function_args = json.loads(response.output[0].arguments)
  if 'date' not in function_args:
      function_args['date'] = date.today().isoformat()

  return function_args

if __name__ ==  "__main__":
  # user_query = "What is the price of Indian Oil Corporation 10th Dec 2024?"
  # user_query = "What is the price of State Bank of India BSE on 10 Jan 2025 ?"
  # user_query = "What is the price of State Bank of India NSE on 10 Dec 2024 ?"
  # user_query = "stock price of  LIC of India  on 5 Oct 2024?"
  # user_query = "Apple Inc price on 3rd Apr 2025"
  user_query = "Indian Oil Corporation price on 3rd Apr 2025"
  stock_data = get_stock_details(user_query)
  print(stock_data)

  price_data = get_stock_price(stock_data['ticker_symbol'], stock_data['date'])
  print(price_data)


[*********************100%***********************]  1 of 1 completed

{'ticker_symbol': 'IOC.NS', 'date': '2025-04-03'}
2025-04-03
INR

--- Yahoo Finance Data for IOC.NS in INR on 2025-04-03 ---
{'price': Price            Close        High         Low        Open   Volume
Ticker          IOC.NS      IOC.NS      IOC.NS      IOC.NS   IOC.NS
Date                                                               
2025-04-03  131.059998  133.199997  129.880005  130.100006  9909642, 'ticker_symbol': 'IOC.NS', 'currency': 'INR'}


In [ ]:
import yfinance as yf
from datetime import date, timedelta
start_date_prev = '2025-04-17'

def is_weekend(d):
  return d.weekday() >= 5

def get_stock_price(ticker_symbol, start_date_prev):
  start_date_prev = date.fromisoformat(start_date_prev)
  while is_weekend(start_date_prev):
    start_date_prev = start_date_prev - timedelta(days=1)

  print(start_date_prev)
  end_date_prev = start_date_prev + timedelta(days=1)

  stock = yf.Ticker(ticker_symbol)
  try:
      currency = stock.info.get("currency", "Unknown Currency")
  except Exception as e:
      currency = "Unknown"
  print(currency)

  try:
      stock_data_prev = yf.download(ticker_symbol, start=start_date_prev, end=end_date_prev)
      if not stock_data_prev.empty:
          print(f"\n--- Yahoo Finance Data for {ticker_symbol} in {currency} on {start_date_prev.isoformat()} ---")
          response = { "price":stock_data_prev,"ticker_symbol":ticker_symbol, "currency": currency }
          return response
      else:
          return(f"\nNo data found for {start_date_prev.isoformat()}.")
  except Exception as e:
      print(f"\nAn error occurred while fetching previous trading day data: {e}")


get_stock_price("AAPL", start_date_prev)


2025-04-17


[*********************100%***********************]  1 of 1 completed

USD

--- Yahoo Finance Data for AAPL in USD on 2025-04-17 ---


{'price': Price            Close        High         Low        Open    Volume
 Ticker            AAPL        AAPL        AAPL        AAPL      AAPL
 Date                                                                
 2025-04-17  196.979996  198.830002  194.419998  197.199997  52164700,
 'ticker_symbol': 'AAPL',
 'currency': 'USD'}